In [1]:
import torch
import json
from tqdm import tqdm
import os
import gc
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
from vllm import LLM, SamplingParams, LLMEngine
from datasets import load_dataset
import csv
from torch.utils.data import Dataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Set environment variable to help with memory fragmentation.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

/home/anamikaghosh_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-06 21:26:36,565	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


cuda


In [2]:
def clean_output(text,keyword):
    # keyword = "Answer:"
    index = text.rfind(keyword)  # Find the last occurrence of "Answer:"
    if index != -1:
        return text[index + len(keyword):].strip()  
    return text

In [3]:
# class ExplanationGeneratorLama:
#     def __init__(self, model_name, max_new_tokens=500):
#         # self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         # self.tokenizer.padding_side = "left"
        
#         self.max_new_tokens = max_new_tokens
#         self.sampling_params = SamplingParams(
#             temperature=0.7,
#             top_p=0.95,
#             max_tokens=max_new_tokens
#         )
#         self.llm = LLM(
#             model=model_name,
#             dtype="half"  # using half precision for GPU
#         )
#         # self.engine = LLMEngine(model_name, sampling_params=self.sampling_params)

#     def generate_explanations_batch(self, entries, max_new_tokens=500):
#         # Create prompts by combining each entry with each prompt template.
        
#         prompts = []
        
#         for entry in entries:
#             prompt_templates = [
#                 f"Doc string: {entry['doc']}\n"
#                 f"Code snippet: {entry['code']}\n"
#                 "Instruction: Provide a concise explanation of what the above doc and code mean. "
#                 "Generate strictly less than 100 words in total. Please give the output just as text only. Do not return anything else.\n"
#                 "Answer: \n"
#                 , 

#                 f"Doc string: {entry['doc']}\n"
#                 f"Code snippet: {entry['code']}\n"
#                 "Instruction: Provide a detailed line-by-line explanation of this code snippet, describing the purpose and functionality of each statement, function, and control structure. "
#                 "Please give the output just as text only. Do not return anything else.\n"
#                 "Answer: \n"
#                 ,

#                 f"Doc string: {entry['doc']}\n"
#                 f"Code snippet: {entry['code']}\n"
#                 "Instruction: Summarize what this code snippet does in simple, non-technical language, focusing on its overall purpose and key operations for someone with little programming experience. "
#                 "Please give the output just as text only. Do not return anything else.\n"
#                 "Answer: \n"
#                 ,

#                 f"Doc string: {entry['doc']}\n"
#                 f"Code snippet: {entry['code']}\n"
#                 "Instruction: Generate an explanation of the code snippet in such a way that it can regenerate the code based on this explanation. "
#                 "Please give the output just as text only. Do not return anything else.\n"
#                 "Answer: \n"
#                 ,

#                 f"Doc string - entry['doc'] : {entry['doc']}\n" \
#                 f"Code snippet - entry['code'] : {entry['code']}\n" \
#                 "Instruction: Explain how the code snippet in entry['code'] implements or achieves the functionality described in the doc string in entry['doc']. Please provide the explanation as text only without any additional content.\n" \
#                 "Answer: \n"
#             ]
            
#             for template in prompt_templates:
#                 prompt = (
#                     f"Doc string: {entry['doc']}\n"
#                     f"Code snippet: {entry['code']}\n"
#                     f"{template}\n"
#                     "Answer: \n"
#                 )
#                 prompts.append(prompt)
                
#         # results = self.engine.generate(prompts)
#         results = self.llm.generate(prompts, self.sampling_params)
#         explanations = [result.outputs[0].text for result in results]
        
#         # Regroup explanations by entry.
#         n_prompts = len(prompt_templates)
#         grouped_explanations = []
#         for i in range(0, len(explanations), n_prompts):
#             grouped_explanations.append(explanations[i:i+n_prompts])
            
#         return grouped_explanations

In [4]:
# if __name__ == "__main__":
#     csv_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/data_preprocessing/CodeSearchNet_Python_valid.csv"  # change this to your CSV input path
#     output_csv_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/explanations/CodeSearchNet_Python_valid_vllm.csv"
    
#     # Load the CSV file into a DataFrame
#     df = pd.read_csv(csv_path)
#     print("Data loaded from CSV")
    

#     models_dict = {
#         "deepseek": "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa",
#         'granite': "/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f"
#     }
    
    
#     batch_size = 200
    
#     for model_key, model_path in tqdm(models_dict.items(), desc="Processing models"):
#         print(f"\nProcessing model {model_key}...")
#         generator = ExplanationGeneratorLama(model_path)
#         if hasattr(generator, 'model'):
#             generator.model.eval()
        
#         # Process the DataFrame in batches
#         for i in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
#             batch_entries = df.iloc[i:i+batch_size][["corpus_id", "query_id", "doc", "code"]].to_dict("records")
            
#             # Wrap inference in a no_grad context to prevent gradient computations.
#             with torch.no_grad():
#                 batch_explanations = generator.generate_explanations_batch(batch_entries)
            
#             for j, explanation_variants in enumerate(batch_explanations):
#                 for idx, raw_text in enumerate(explanation_variants):
                    
#                     df.loc[i+j, f'explanation_{model_key}_{idx+1}'] = raw_text
            
#             # torch.cuda.empty_cache()
#             # gc.collect()
        
#         del generator
#         # torch.cuda.empty_cache()
#         # gc.collect()
#         df.to_csv(output_csv_path, index=False)
    
#     # df.to_csv(output_csv_path, index=False)
#     print(f"\nExplanations from all models have been saved to {output_csv_path}")


## Read data batch by batch

In [ ]:
# temp = {'Unnamed: 0': 0, 'query_id': 'q251820', 'corpus_id': 'c251820', 'score': 1, 'doc': 'Save model to a pickle located at `path`', 'code': 'def save_act(self, path=None):\n        """Save model to a pickle located at `path`"""\n        if path is None:\n            path = os.path.join(logger.get_dir(), "model.pkl")\n\n        with tempfile.TemporaryDirectory() as td:\n            save_variables(os.path.join(td, "model"))\n            arc_name = os.path.join(td, "packed.zip")\n            with zipfile.ZipFile(arc_name, \'w\') as zipf:\n                for root, dirs, files in os.walk(td):\n                    for fname in files:\n                        file_path = os.path.join(root, fname)\n                        if file_path != arc_name:\n                            zipf.write(file_path, os.path.relpath(file_path, td))\n            with open(arc_name, "rb") as f:\n                model_data = f.read()\n        with open(path, "wb") as f:\n            cloudpickle.dump((model_data, self._act_params), f)'}
# pd.DataFrame(temp)

In [ ]:
# Your custom class
class ExplanationGeneratorLama:
    def __init__(self, model_name, max_new_tokens=500):
        self.max_new_tokens = max_new_tokens
        self.sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.95,
            max_tokens=max_new_tokens
        )
        self.llm = LLM(model=model_name, dtype="half")  # GPU half precision

    def generate_explanations_batch(self, entries):
        prompts = []
        for entry in entries:
            code = entry['code']
            prompt_templates = [
            
            f"Code snippet: {code}\n"
            "Instruction: Provide a concise explanation of what the above code mean. "
            "Generate strictly less than 100 words in total. Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n"
            , 

            
            f"Code snippet: {code}\n"
            "Instruction: Provide a detailed line-by-line explanation of this code snippet, describing the purpose and functionality of each statement, function, and control structure. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n"
            ,

            
            f"Code snippet: {code}\n"
            "Instruction: Summarize what this code snippet does in simple, non-technical language, focusing on its overall purpose and key operations for someone with little programming experience. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n"
            ,

            
            f"Code snippet: {code}\n"
            "Instruction: Generate an explanation of the code snippet in such a way that it can regenerate the code based on this explanation. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n"
            ,

        
            f"Code snippet - entry['code'] : {code}\n" \
            "Instruction: Explain how the code snippet in entry['code'] implements. Please provide the explanation as text only without any additional content.\n" \
            "Answer: \n"
        ]
            prompts.extend(prompt_templates)

        results = self.llm.generate(prompts, self.sampling_params)
        outputs = [res.outputs[0].text for res in results]

        # Group by original entry
        grouped = []
        for i in range(0, len(outputs), 5):
            grouped.append(outputs[i:i+5])

        return grouped


# === CONFIG ===
csv_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/data_preprocessing/CodeSearchNet_Python_valid.csv'
output_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/data_preprocessing/output.csv'
checkpoint_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/data_preprocessing/checkpoint.txt'
model_path = "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa"
model = 'deepseek'
batch_size = 128  # adjust to your GPU memory

# === HELPERS ===
def load_checkpoint():
    return int(open(checkpoint_path).read()) if os.path.exists(checkpoint_path) else 0

def save_checkpoint(line_num):
    with open(checkpoint_path, 'w') as f:
        f.write(str(line_num))

def write_header_if_needed(output_path, fieldnames):
    if not os.path.exists(output_path) or os.stat(output_path).st_size == 0:
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()

# === MAIN ===
def main():
    dataset = load_dataset('csv', data_files=csv_path, split='train', streaming=True)
    print('Dataset loaded')

    start_line = load_checkpoint()
    print(f'Starting from line {start_line}')

    generator = ExplanationGeneratorLama(model_path)
    if hasattr(generator, 'model'):
        generator.model.eval()
    print('Model loaded')

    buffer = []
    line_num = 0
    written_header = False
    total_len = sum(1 for _ in dataset)

    with open(output_path, 'a', newline='', encoding='utf-8') as outfile:
        writer = None

        for row in tqdm(dataset, desc="Streaming dataset", total=total_len):
            line_num += 1
            if line_num <= start_line:
                continue

            buffer.append(row)

            if len(buffer) == batch_size:
                with torch.no_grad():
                    all_explanations = generator.generate_explanations_batch(buffer)

                for entry, explanations in zip(buffer, all_explanations):
                    for idx, explanation in enumerate(explanations):
                        entry[f'explanation_deepseek_{idx+1}'] = explanation

                    # Setup writer on first pass
                    if writer is None:
                        fieldnames = list(entry.keys())
                        write_header_if_needed(output_path, fieldnames)
                        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

                    writer.writerow(entry)

                outfile.flush()
                save_checkpoint(line_num)
                buffer.clear()

        # Final batch (if any)
        if buffer:
            with torch.no_grad():
                all_explanations = generator.generate_explanations_batch(buffer)

            for entry, explanations in zip(buffer, all_explanations):
                for idx, explanation in enumerate(explanations):
                    entry[f'explanation_{model}_{idx+1}'] = explanation

                if writer is None:
                    fieldnames = list(entry.keys())
                    write_header_if_needed(output_path, fieldnames)
                    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

                writer.writerow(entry)

            outfile.flush()
            save_checkpoint(line_num)

    print(f"✅ Finished processing. Output saved to {output_path}")

# === RUN ===
if __name__ == "__main__":
    main()


Dataset loaded
Starting from line 0
INFO 04-06 21:26:39 __init__.py:207] Automatically detected platform cuda.
WARNING 04-06 21:26:39 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 04-06 21:26:50 config.py:549] This model supports multiple tasks: {'generate', 'score', 'embed', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 04-06 21:26:50 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 04-06 21:26:50 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 04-06 21:26:50 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa', speculative_config=None, tokenizer='/datas

[W406 21:26:52.883931974 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.45it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.45it/s]



INFO 04-06 21:26:53 model_runner.py:1115] Loading model weights took 3.3414 GB
INFO 04-06 21:26:54 worker.py:267] Memory profiling takes 0.44 seconds
INFO 04-06 21:26:54 worker.py:267] the current vLLM instance can use total_gpu_memory (44.52GiB) x gpu_memory_utilization (0.90) = 40.07GiB
INFO 04-06 21:26:54 worker.py:267] model weights take 3.34GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 35.25GiB.
INFO 04-06 21:26:54 executor_base.py:111] # cuda blocks: 82516, # CPU blocks: 9362
INFO 04-06 21:26:54 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 10.07x
INFO 04-06 21:26:56 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:12<00:00,  2.90it/s]

INFO 04-06 21:27:08 model_runner.py:1562] Graph capturing finished in 12 secs, took 0.21 GiB
INFO 04-06 21:27:08 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 15.12 seconds


Model loaded


Streaming dataset:   7%|▋         | 999/13788 [02:46<35:26,  6.02it/s]  


KeyboardInterrupt: 

## PyTorch Dataloader (ignore) - Slower

In [3]:
class ExplanationGeneratorLama:
    def __init__(self, model_name, max_new_tokens=500):
        self.max_new_tokens = max_new_tokens
        self.sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.95,
            max_tokens=max_new_tokens
        )
        self.llm = LLM(
            model=model_name,
            dtype="half"  # using half precision for GPU
        )

    def generate_explanations_batch(self, entries, max_new_tokens=500):
        prompts = []
        # List of prompt templates for generating various explanations
        prompt_templates = [
            f"Doc string: {entries['doc']}\n"
            f"Code snippet: {entries['code']}\n"
            "Instruction: Provide a concise explanation of what the above doc and code mean. "
            "Generate strictly less than 100 words in total. Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n",

            f"Doc string: {entries['doc']}\n"
            f"Code snippet: {entries['code']}\n"
            "Instruction: Provide a detailed line-by-line explanation of this code snippet, describing the purpose and functionality of each statement, function, and control structure. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n",

            f"Doc string: {entries['doc']}\n"
            f"Code snippet: {entries['code']}\n"
            "Instruction: Summarize what this code snippet does in simple, non-technical language, focusing on its overall purpose and key operations for someone with little programming experience. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n",

            f"Doc string: {entries['doc']}\n"
            f"Code snippet: {entries['code']}\n"
            "Instruction: Generate an explanation of the code snippet in such a way that it can regenerate the code based on this explanation. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n",

            f"Doc string - entry['doc'] : {entries['doc']}\n"
            f"Code snippet - entry['code'] : {entries['code']}\n"
            "Instruction: Explain how the code snippet in entry['code'] implements or achieves the functionality described in the doc string in entry['doc']. Please provide the explanation as text only without any additional content.\n"
            "Answer: \n"
        ]
        
        # Create a prompt for each template
        for template in prompt_templates:
            prompt = (
                f"Doc string: {entries['doc']}\n"
                f"Code snippet: {entries['code']}\n"
                f"{template}\n"
                "Answer: \n"
            )
            prompts.append(prompt)
                
        results = self.llm.generate(prompts, self.sampling_params)
        explanations = [result.outputs[0].text for result in results]
        return explanations

In [4]:
# File paths
csv_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/data_preprocessing/CodeSearchNet_Python_valid.csv'
output_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/data_preprocessing/output.csv'
checkpoint_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/data_preprocessing/checkpoint.txt'

# Checkpoint functions to resume processing
def load_checkpoint():
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as f:
            return int(f.read().strip())
    return 0

def save_checkpoint(line_num):
    with open(checkpoint_path, 'w') as f:
        f.write(str(line_num))

# Custom PyTorch Dataset to read from the DataFrame
class CodeExplanationDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        return row.to_dict()

# Process a batch of rows from the DataLoader
def process_batch(batch, generator):
    processed_rows = []
    # batch is a dictionary where each key maps to a list of values (one per row in the batch)
    batch_size = len(batch['doc'])
    for i in range(batch_size):
        entry = {
            'doc': batch['doc'][i],
            'code': batch['code'][i]
        }
        explanations = generator.generate_explanations_batch(entry)
        # Create a new dictionary for the processed row that includes original fields
        row_dict = {key: batch[key][i] for key in batch}
        for idx, explanation in enumerate(explanations):
            row_dict[f'explanation_deepseek_{idx+1}'] = explanation
        processed_rows.append(row_dict)
    return processed_rows

In [5]:
def main():
    # Read the entire CSV file using pandas
    df = pd.read_csv(csv_path)
    
    # Load checkpoint and skip already processed rows
    start_line = load_checkpoint()
    if start_line > 0:
        df = df.iloc[start_line:]
    
    dataset = CodeExplanationDataset(df)
    # Adjust the batch size as needed (e.g., 4)
    batch_size = 10
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model_path = "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa"
    generator = ExplanationGeneratorLama(model_path)
    if hasattr(generator, 'model'):
        generator.model.eval()
    
    with open(output_path, 'a', newline='', encoding='utf-8') as outfile:
        writer = None
        processed_count = start_line
        
        for batch in tqdm(dataloader):
            processed_rows = process_batch(batch, generator)
            for row in processed_rows:
                processed_count += 1
                # Initialize CSV writer if it has not been created yet
                if writer is None:
                    fieldnames = list(row.keys())
                    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
                    # Write header if output file is empty
                    if os.stat(output_path).st_size == 0:
                        writer.writeheader()
                writer.writerow(row)
                save_checkpoint(processed_count)
                print(f'Processed row no. {processed_count}')

In [ ]:
if __name__ == '__main__':
    main()

INFO 04-06 20:39:17 __init__.py:207] Automatically detected platform cuda.
WARNING 04-06 20:39:17 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 04-06 20:39:28 config.py:549] This model supports multiple tasks: {'embed', 'reward', 'classify', 'score', 'generate'}. Defaulting to 'generate'.
WARNING 04-06 20:39:28 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 04-06 20:39:28 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 04-06 20:39:28 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa', speculative_config=None, tokenizer='/datasets/ai/deepseek/hub/models--deepseek

[W406 20:39:30.563524972 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.47it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.47it/s]



INFO 04-06 20:39:31 model_runner.py:1115] Loading model weights took 3.3414 GB
INFO 04-06 20:39:31 worker.py:267] Memory profiling takes 0.44 seconds
INFO 04-06 20:39:31 worker.py:267] the current vLLM instance can use total_gpu_memory (44.52GiB) x gpu_memory_utilization (0.90) = 40.07GiB
INFO 04-06 20:39:31 worker.py:267] model weights take 3.34GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 35.25GiB.
INFO 04-06 20:39:31 executor_base.py:111] # cuda blocks: 82516, # CPU blocks: 9362
INFO 04-06 20:39:31 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 10.07x
INFO 04-06 20:39:34 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:12<00:00,  2.91it/s]

INFO 04-06 20:39:46 model_runner.py:1562] Graph capturing finished in 12 secs, took 0.21 GiB
INFO 04-06 20:39:46 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 15.08 seconds



  0%|          | 1/1375 [00:33<12:45:41, 33.44s/it]

Processed row no. 45
Processed row no. 46
Processed row no. 47
Processed row no. 48
Processed row no. 49
Processed row no. 50
Processed row no. 51
Processed row no. 52
Processed row no. 53
Processed row no. 54


  0%|          | 2/1375 [01:05<12:30:04, 32.78s/it]

Processed row no. 55
Processed row no. 56
Processed row no. 57
Processed row no. 58
Processed row no. 59
Processed row no. 60
Processed row no. 61
Processed row no. 62
Processed row no. 63
Processed row no. 64


Processed prompts: 100%|██████████| 5/5 [00:03<00:00,  1.51it/s, est. speed input: 1238.73 toks/s, output: 459.18 toks/s]


In [ ]:
# 3 sec per row (or) example using dataloader